<a href="https://colab.research.google.com/github/rotom303/Final_Project/blob/main/Final_Algorithm_Submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Run all cells in this notebook. Before running, ensure that the current working directory (i.e. the directory where the drive is mounted) contains the 'models' folder and the 'raw_training_dat.mat' file. The code loads the models from the 'models' folder and uses the 'raw_training_data.mat' file to normalize the testing data. It assumes that the 'models' folder and the 'raw_training_data.mat' and located in its current working directory, along with the 'truetest_data.mat' file.

## The code outputs predictions.mat, which contains a variable predicted_dg, which contains our predictions on the hidden test set.

In [ ]:
#Set up the notebook environment
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import scipy
import random
from scipy.stats import pearsonr
from scipy import signal as sig
from sklearn.ensemble import GradientBoostingRegressor
from xgboost.sklearn import XGBRegressor

from sklearn.preprocessing import StandardScaler
from mlxtend.regressor import StackingCVRegressor
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import train_test_split, KFold
from sklearn.svm import SVR
from numpy.linalg import inv

NUM_FINGERS = 5

In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

#%cd drive/MyDrive/Penn/'Spring 2023'/'BE 5210'/Project/Part_2

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'content/drive/MyDrive/Penn/Spring 2023/BE 5210/Project/Part_2'
/root


In [ ]:
import scipy.io as sio

proj_data=sio.loadmat('raw_training_data.mat')

#splitting into testing and training sets
train_data_proportion = 0.7
#patient 1

random.seed(123)

#glove
total_p1_glove=proj_data['train_dg'][0][0]
training_rows = list(range(0, int(len(total_p1_glove)*train_data_proportion)))
training_p1_glove=total_p1_glove[training_rows,:] #60% of samples in training
testing_p1_glove=np.delete(total_p1_glove, training_rows, axis=0) #40% of samples in testing

#ecog
total_p1_ecog=proj_data['train_ecog'][0][0]
# total_p1_ecog=np.delete(total_p1_ecog, 54, 1)
training_rows = list(range(0, int(len(total_p1_ecog)*train_data_proportion)))
training_p1_ecog=total_p1_ecog[training_rows,:] #60% of samples in training
testing_p1_ecog=np.delete(total_p1_ecog, training_rows, axis=0) #60% of samples in training

#patient 2

#glove
total_p2_glove=proj_data['train_dg'][1][0]
training_rows = list(range(0, int(len(total_p2_glove)*train_data_proportion)))
training_p2_glove=total_p2_glove[training_rows,:] #60% of samples in training
testing_p2_glove=np.delete(total_p2_glove, training_rows, axis=0) #60% of samples in training
#ecog
total_p2_ecog=proj_data['train_ecog'][1][0]
# total_p2_ecog=np.delete(total_p2_ecog, [20, 37], 1)
training_rows = list(range(0, int(len(total_p2_ecog)*train_data_proportion)))
training_p2_ecog=total_p2_ecog[training_rows,:] #60% of samples in training
testing_p2_ecog=np.delete(total_p2_ecog, training_rows, axis=0) #60% of samples in training

#patient 3

#glove
total_p3_glove=proj_data['train_dg'][2][0]
training_rows = list(range(0, int(len(total_p3_glove)*train_data_proportion)))
training_p3_glove=total_p3_glove[training_rows,:] #60% of samples in training
testing_p3_glove=np.delete(total_p3_glove, training_rows, axis=0) #60% of samples in training
#ecog
total_p3_ecog=proj_data['train_ecog'][2][0]
training_rows = list(range(0, int(len(total_p3_ecog)*train_data_proportion)))
training_p3_ecog=total_p3_ecog[training_rows,:] #60% of samples in training
testing_p3_ecog=np.delete(total_p3_ecog, training_rows, axis=0) #60% of samples in training

In [ ]:
def filter_data(raw_eeg, fs=1000, cutoffs=[75, 200]):
  """
  Write a filter function to clean underlying data.
  Filter type and parameters are up to you. Points will be awarded for reasonable filter type, parameters and application.
  Please note there are many acceptable answers, but make sure you aren't throwing out crucial data or adversly
  distorting the underlying data!

  Input: 
    raw_eeg (samples x channels): the raw signal
    fs: the sampling rate (1000 for this dataset)
  Output: 
    clean_data (samples x channels): the filtered signal
  """
  number_of_channels = np.shape(raw_eeg)[1] #number of channels
  filteredData = np.zeros(np.shape(raw_eeg)); #filtered data output

  #butterworth filter of 4th order
  sos = sig.butter(4, cutoffs, 'bandpass', analog=False, fs=fs, output='sos'); # returns filter coefficients

  #for each channel
  for chanInd in np.arange(number_of_channels):
    # subtract mean from each datapoint
    currFilt = raw_eeg[:, chanInd] - np.mean(raw_eeg[:, chanInd]);
    currFilt = sig.sosfiltfilt(sos, currFilt) # forward-backward digital filter using cascaded second-order sections                                
    filteredData[:, chanInd] = currFilt
  return filteredData

In [ ]:
def NumWins(x, winLen, winDisp, fs):

    # Calculate the length of the signal in samples
    xLen = len(x)
    # Calculate the number of windows
    windows = ((len(x)/fs)/ winDisp) - (winLen / winDisp) + (winDisp / winDisp) - ((((len(x)/fs) - winLen + winDisp) % winDisp)/winDisp)
    #windows = ((len(x)/fs)/ winDisp) - (winLen / winDisp) + (winDisp / winDisp)
    return int(windows)

In [ ]:
def line_length(signal):
    return np.sum(np.absolute(np.ediff1d(signal))) 

def zero_crossings(x):
  xbar=np.mean(x)
  number_of_crossings = 0
  for i in range(1,len(x)):
    if (x[i-1]-xbar) > 0 and (x[i]-xbar) < 0:
      number_of_crossings += 1
    elif (x[i-1]-xbar) < 0 and (x[i]-xbar) > 0:
      number_of_crossings += 1
  return number_of_crossings

def energy(signal):
  return np.sum(np.square(signal))

def area(signal):
  return np.sum(np.absolute(signal))

def time_avg(signal):
  return np.mean(signal)

from numpy.fft import fft, ifft

def band_power(signal,fs,low,high):
  X = fft(signal)
  N = len(X)
  n = np.arange(N)
  T = N/fs #sampling rate=1000
  freq = n/T 
  power_spectrum=np.abs(X)

  # Find values in frequency vector corresponding to input band
  index_band = np.logical_and(freq >= low, freq <= high)
  #average frequency domain magnitude
  avg_mag=np.mean(power_spectrum[index_band])

  return avg_mag

In [ ]:
def get_features(filtered_window, fs=1000):
  """
    Write a function that calculates features for a given filtered window. 
    Feel free to use features you have seen before in this class, features that
    have been used in the literature, or design your own!

    Input: 
      filtered_window (window_samples x channels): the window of the filtered ecog signal 
      fs: sampling rate
    Output:
      features (channels x num_features): the features calculated on each channel for the window
  """
  #print(np.shape(filtered_window))
  [window_samples,num_channels]=np.shape(filtered_window)

  features=np.empty((num_channels,5))

  for cc in range(num_channels):

    curr_window=filtered_window[:,cc] 

    #features[cc,0]=line_length(curr_window)
    #features[cc,0]=area(curr_window)
    features[cc,0]=energy(curr_window) # Energy in signal window
    #features[cc,3]=zero_crossings(curr_window)

    features[cc,1]=band_power(curr_window,1000,8,12) # average frequency-domain magnitude in alpha frequency band
    #features[cc,2]=band_power(curr_window,1000,13,30) # average frequency-domain magnitude in beta frequency band
    features[cc,2]=band_power(curr_window,1000,75,95) # average frequency-domain magnitude in low gamma frequency band
    features[cc,3]=band_power(curr_window,1000,96,115) # average frequency-domain magnitude in high gamma frequency band
    features[cc,4]=time_avg(curr_window) #Average time-domain voltage

  return features

In [ ]:
def get_windowed_feats(raw_ecog, fs, window_length, window_overlap):
  """
    Write a function which processes data through the steps of filtering and
    feature calculation and returns features. Points will be awarded for completing
    each step appropriately (note that if one of the functions you call within this script
    returns a bad output, you won't be double penalized). Note that you will need
    to run the filter_data and get_features functions within this function. 

    Inputs:
      raw_eeg (samples x channels): the raw signal
      fs: the sampling rate (1000 for this dataset)
      window_length: the window's length
      window_overlap: the window's overlap
    Output: 
      all_feats (num_windows x (channels x features)): the features for each channel for each time window
        note that this is a 2D array. 
  """
  clean_data=filter_data(raw_ecog, fs)
  [num_samples,num_channels]=np.shape(clean_data)

  num_windows = NumWins(clean_data[:,0], window_length,window_overlap, fs) #calculate number of windows and remaining time

  #convert everything to units of samples
  num_samples = len(clean_data)
  winLen_samples=round(window_length*fs) #window length in samples
  winDisp_samples=round(window_overlap*fs) #window displacement in samples
  
  # empty list to store features
  feature_vector = []
  # The end of the first window is the last sample
  window_end = num_samples
  # List to store tuples containing start and end indices for each slice

  for i in range(round(num_windows)):

    # Compute the start of the window using the end of the window
    window_start = window_end - winLen_samples
    # If the number of samples left is not sufficient, break out of the loop
    if window_start < 0:
        break
    
    signal_window=clean_data[window_start:window_end,:] #signal in that window
    
    # store feature of that window
    feature_vector.append(get_features(signal_window, fs).flatten('F'))

    #set new ending index of window
    window_end -= winDisp_samples 
  
  # Reverse at the end since we are looking at the windows backwards
  feature_vector.reverse()


  return feature_vector

In [ ]:
def create_R_matrix(features, N_wind):
  """
  Write a function to calculate the R matrix

  Input:
    features (samples (number of windows in the signal) x channels x features): 
      the features you calculated using get_windowed_feats
    N_wind: number of windows to use in the R matrix

  Output:
    R (samples x (N_wind*channels*features))
  """
  padded_features = np.copy(features)
  for i in list(range(N_wind-2, -1, -1)):
      a = features[i]
      padded_features = np.vstack([a, padded_features])
  samples = len(features)   # number of rows = number of windows

  R = np.zeros((samples, 1+(N_wind*len(features[0,:]))))  # len(features[0,:]) = (num of features)*(num of channels)
  lst = np.array(list(range(1, 1+N_wind)))
  R[:, 0] = 1

  for i in range(len(features[0,:])):   # goes thru each column of the features matrix
    for j in range(len(lst)):
        x = lst[j]
        R[:, x] = padded_features[j : (len(padded_features)-(N_wind-1-j)), i]
    lst = lst + N_wind
  return R

In [ ]:
def standardize_train_test(feature_matrix,test_feature_matrix):

  [windows,feats]=np.shape(feature_matrix)
  [windows_test,feats_test]=np.shape(test_feature_matrix)
  normFeats=np.empty((windows,feats))
  testnormFeats=np.empty((windows_test,feats_test))

  for i in range(feats):
      curr_train = feature_matrix[:,i]
      mean_train=np.mean(curr_train)
      std_train=np.std(curr_train)
      normFeats[:,i]=(curr_train-mean_train)/std_train
      curr_test = test_feature_matrix[:,i]
      mean_test=np.mean(curr_test)
      std_test=np.std(curr_test)
      testnormFeats[:,i]=(curr_test-mean_train)/std_train #normalize test features using training mean/std

  return normFeats,testnormFeats

In [ ]:
def smooth(input, smoothSize):
  toConv = sig.gaussian(smoothSize, 3)/smoothSize;
  return np.convolve(input, toConv, mode='same')

## Loading In truetest_data.mat

In [ ]:
truetest_data = scipy.io.loadmat('truetest_data.mat')['truetest_data']
p1_truetest_ecog=truetest_data[0][0]
p2_truetest_ecog=truetest_data[1][0]
p3_truetest_ecog=truetest_data[2][0]
p1_ecog=p1_truetest_ecog[:,np.arange(62)!=54] #remove channel 55
p2_ecog=p2_truetest_ecog[:,np.logical_and(np.arange(48)!=20,np.arange(48)!=37)] #remove channel 21 and 38
p3_ecog=p3_truetest_ecog

In [ ]:
window_length=100e-3
window_overlap=50e-3
fs=1000

p1_Feats_l=get_windowed_feats(p1_truetest_ecog, fs, window_length, window_overlap)
p2_Feats_l=get_windowed_feats(p2_truetest_ecog, fs, window_length, window_overlap)
p3_Feats_l=get_windowed_feats(p3_truetest_ecog, fs, window_length, window_overlap)
p1_Feats_l=np.array(p1_Feats_l)
p2_Feats_l=np.array(p2_Feats_l)
p3_Feats_l=np.array(p3_Feats_l)

In [ ]:
p1_allFeats = get_windowed_feats(total_p1_ecog, fs, window_length, window_overlap)
p1_allFeats = np.array(p1_allFeats)
p2_allFeats = get_windowed_feats(total_p2_ecog, fs, window_length, window_overlap)
p2_allFeats = np.array(p2_allFeats)
p3_allFeats = get_windowed_feats(total_p3_ecog, fs, window_length, window_overlap)
p3_allFeats = np.array(p3_allFeats)

In [ ]:
p1_normalized_all, p1_normalized_leader = standardize_train_test(p1_allFeats, p1_Feats_l)
p2_normalized_all, p2_normalized_leader = standardize_train_test(p2_allFeats, p2_Feats_l)
p3_normalized_all, p3_normalized_leader = standardize_train_test(p3_allFeats, p3_Feats_l)
R_p1_leader=create_R_matrix(p1_normalized_leader,3)
R_p2_leader=create_R_matrix(p2_normalized_leader,3)
R_p3_leader=create_R_matrix(p3_normalized_leader,3)

In [ ]:
samples,channels=np.shape(p1_truetest_ecog)
truetest_pred_3=np.zeros((samples, NUM_FINGERS))
truetest_pred_2=np.zeros((samples, NUM_FINGERS))
truetest_pred_1=np.zeros((samples, NUM_FINGERS))

## Load In Models

In [ ]:
with open('models/p1_f1.pkl', 'rb') as f:
    fin_p1_f1 = pickle.load(f)

with open('models/p1_f2.pkl', 'rb') as f:
    fin_p1_f2 = pickle.load(f)

with open('models/p1_f3.pkl', 'rb') as f:
    fin_p1_f3 = pickle.load(f)

with open('models/p1_f4.pkl', 'rb') as f:
    fin_p1_f4 = pickle.load(f)

with open('models/p1_f5.pkl', 'rb') as f:
    fin_p1_f5 = pickle.load(f)

with open('models/p2_f1.pkl', 'rb') as f:
    fin_p2_f1 = pickle.load(f)

with open('models/p2_f2.pkl', 'rb') as f:
    fin_p2_f2 = pickle.load(f)

with open('models/p2_f3.pkl', 'rb') as f:
    fin_p2_f3 = pickle.load(f)

with open('models/p2_f4.pkl', 'rb') as f:
    fin_p2_f4 = pickle.load(f)

with open('models/p2_f5.pkl', 'rb') as f:
    fin_p2_f5 = pickle.load(f)

with open('models/p3_f1.pkl', 'rb') as f:
    fin_p3_f1 = pickle.load(f)

with open('models/p3_f2.pkl', 'rb') as f:
    fin_p3_f2 = pickle.load(f)

with open('models/p3_f3.pkl', 'rb') as f:
    fin_p3_f3 = pickle.load(f)

with open('models/p3_f4.pkl', 'rb') as f:
    fin_p3_f4 = pickle.load(f)

with open('models/p3_f5.pkl', 'rb') as f:
    fin_p3_f5 = pickle.load(f)

In [ ]:
fin_model_list = [[fin_p1_f1, fin_p1_f2, fin_p1_f3, fin_p1_f4, fin_p1_f5],
              [fin_p2_f1, fin_p2_f2, fin_p2_f3, fin_p2_f4, fin_p2_f5],
              [fin_p3_f1, fin_p3_f2, fin_p3_f3, fin_p3_f4, fin_p3_f5]]

### Predict and Interpolate

In [ ]:
from scipy.interpolate import CubicSpline

for i in range(5):
    print("i =", i)
    # Patient 1
    y_pred_test3 = fin_model_list[2][i].predict(R_p3_leader)
    smoothed3=smooth(y_pred_test3,20)
    cs = CubicSpline(np.arange(smoothed3.shape[0]), smoothed3)
    new_x = np.linspace(0, smoothed3.shape[0]-1, p3_truetest_ecog.shape[0])
    totalPrediction3 = cs(new_x)
    truetest_pred_3[:,i]=totalPrediction3
    
    # Patient 2
    y_pred_test2 = fin_model_list[1][i].predict(R_p2_leader)
    smoothed2=smooth(y_pred_test2,20)
    cs = CubicSpline(np.arange(smoothed2.shape[0]), smoothed2)
    new_x = np.linspace(0, smoothed2.shape[0]-1, p2_truetest_ecog.shape[0])
    totalPrediction2 = cs(new_x)
    truetest_pred_2[:,i]=totalPrediction2

    # Patient 1
    y_pred_test1 = fin_model_list[0][i].predict(R_p1_leader)
    smoothed1=smooth(y_pred_test1,20)
    cs = CubicSpline(np.arange(smoothed1.shape[0]), smoothed1)
    new_x = np.linspace(0, smoothed1.shape[0]-1, p1_truetest_ecog.shape[0])
    totalPrediction1 = cs(new_x)
    truetest_pred_1[:,i]=totalPrediction1

In [ ]:
import numpy as np
from scipy.io import savemat

predictions = np.zeros((3,1), dtype=object)
predictions[0,0] = truetest_pred_1
predictions[1,0] = truetest_pred_2
predictions[2,0] = truetest_pred_3

#save the array using the right format
savemat('predictions.mat', {'predicted_dg':predictions})